### 1098. Unpopular Books(Median)



Table: Books
```
+----------------+---------+
| Column Name    | Type    |
+----------------+---------+
| book_id        | int     |
| name           | varchar |
| available_from | date    |
+----------------+---------+
```
book_id is the primary key of this table.
 

Table: Orders
```
+----------------+---------+
| Column Name    | Type    |
+----------------+---------+
| order_id       | int     |
| book_id        | int     |
| quantity       | int     |
| dispatch_date  | date    |
+----------------+---------+
```
order_id is the primary key of this table.
book_id is a foreign key to the Books table.

```
    select 
    book_id, name  
    from
    (
        select 
            b.book_id
            , b.name
            , case when datediff(b.available_from, '2019-06-23') < -30 then 1 else 0 end as is_valid
            , sum(case when datediff(o.dispatch_date, '2019-06-23') >= -365 then quantity else 0 end) over(partition by book_id) sum_copyies_valid
        from
            Books b
        left join
            Orders o
        on 
            b.book_id = o.book_id
    ) tmp
    where 
    tmp.is_valid = 1 and tmp.sum_copyies_valid < 10
    group by
    book_id, name  
    order by 
    book_id
```
  




### 1107. New Users Daily Count(Median)



Table: Traffic
```
+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| user_id       | int     |
| activity      | enum    |
| activity_date | date    |
+---------------+---------+
```
There is no primary key for this table, it may have duplicate rows.
The activity column is an ENUM type of ('login', 'logout', 'jobs', 'groups', 'homepage').
 

Write an SQL query to reports for every date within at most 90 days from today, the number of users that logged in for the first time on that date. Assume today is 2019-06-30.

Return the result table in any order.

The query result format is in the following example.

 

Example 1:

Input: 
Traffic table:
```
+---------+----------+---------------+
| user_id | activity | activity_date |
+---------+----------+---------------+
| 1       | login    | 2019-05-01    |
| 1       | homepage | 2019-05-01    |
| 1       | logout   | 2019-05-01    |
| 2       | login    | 2019-06-21    |
| 2       | logout   | 2019-06-21    |
| 3       | login    | 2019-01-01    |
| 3       | jobs     | 2019-01-01    |
| 3       | logout   | 2019-01-01    |
| 4       | login    | 2019-06-21    |
| 4       | groups   | 2019-06-21    |
| 4       | logout   | 2019-06-21    |
| 5       | login    | 2019-03-01    |
| 5       | logout   | 2019-03-01    |
| 5       | login    | 2019-06-21    |
| 5       | logout   | 2019-06-21    |
+---------+----------+---------------+
```
Output: 
```
+------------+-------------+
| login_date | user_count  |
+------------+-------------+
| 2019-05-01 | 1           |
| 2019-06-21 | 2           |
+------------+-------------+
```
Explanation: 
Note that we only care about dates with non zero user count.
The user with id 5 first logged in on 2019-03-01 so he's not counted on 2019-06-21.


```
    select
    activity_date as login_date, count(distinct(user_id)) as user_count
    from
    (
        select
        *
        , min(activity_date) over(partition by user_id) as min_activity_date
        from
        Traffic t
        where 
        activity = "login" 
    ) tmp
    where min_activity_date = activity_date
    group by 
    activity_date
    having 
    datediff(activity_date, '2019-06-30') >= -90
```

### 1112. Highest Grade For Each Student(Median)

Table: Enrollments
```
+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| student_id    | int     |
| course_id     | int     |
| grade         | int     |
+---------------+---------+
(student_id, course_id) is the primary key of this table.
``` 

Write a SQL query to find the highest grade with its corresponding course for each student. In case of a tie, you should find the course with the smallest course_id.

Return the result table ordered by student_id in ascending order.

The query result format is in the following example.


```
select student_id, min(course_id) as course_id, grade
from
(
    select 
        *, max(grade) over(partition by student_id) as max_score 
    from 
        Enrollments 
) tmp
where 
  tmp.grade = tmp.max_score
group by
  student_id, grade
```


### 1126. Active Businesses(Median)

Table: Events
```
+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| business_id   | int     |
| event_type    | varchar |
| occurences    | int     | 
+---------------+---------+
```
(business_id, event_type) is the primary key of this table.
Each row in the table logs the info that an event of some type occurred at some business for a number of times.
 

The average activity for a particular event_type is the average occurences across all companies that have this event.

An active business is a business that has more than one event_type such that their occurences is strictly greater than the average activity for that event.

Write an SQL query to find all active businesses.

Return the result table in any order.

The query result format is in the following example.

```
select
  business_id
from
(
    select
        *,
        avg(occurences) over(partition by event_type)  as avg_occur
    from 
        Events
)t1
where
  t1.occurences > t1.avg_occur
group by 
  business_id
having count(business_id) > 1

```